In [1]:
from stable_baselines3 import  SAC
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor

import gym
import numpy as np
from datetime import datetime

from KBMproject import ATLA
import KBMproject.utilities as utils

from citylearn.data import DataSet

Basic constants

In [2]:
DATASET_NAME = 'citylearn_challenge_2022_phase_2'
SAVE_DIR = 'Models/ATLAc/'
LOG_DIR = 'logs/Phase3/ATLAc/'
VERBOSITY = 0
DEVICE = 'cuda'

The max mean difference represents the largest change between two samples for each feature minus the mean difference. This will be the maximum perturbation size for our adversary. Using the max difference represents the wors case scenario we expect to encounter based on our training data. Because this is derived from the difference between samples, we subtract the mean difference so on average the inter sample change will not exceed the max recorded value. This is our boundary for the adversary's perturbation.
see bline obs analysis.ipynb in the PPO 500 results

In [3]:
MAX_MEAN_DIFF = np.array([0.24977164, 0.24977164, 0.34341758, 0.69515118, 0.04606484,
                        0.04608573, 0.26690566, 0.26690266, 0.2669048 , 0.26690781,
                        0.62865948, 0.62865314, 0.62865568, 0.62865948, 0.52596206,
                        0.52596487, 0.52598294, 0.52596206, 0.75557218, 0.75558416,
                        0.75558188, 0.75557218, 0.28202381, 0.61189055, 0.00253725,
                        0.47459565, 0.0052361 , 0.89720221, 0.89720221, 0.89720221,
                        0.89720221])

MEAN_DIFF = np.array([0.12511418, 0.12511418, 0.18184461, 0.35953119, 0.10637713,
                     0.10636668, 0.15978021, 0.15978171, 0.15978064, 0.15977914,
                     0.36344801, 0.36345118, 0.36344991, 0.36344801, 0.3260062 ,
                     0.3260048 , 0.32599576, 0.3260062 , 0.44802713, 0.44802114,
                     0.44802228, 0.44802713, 0.16781362, 0.36620854, 0.00152669,
                     0.31896562, 0.00326229, 0.52109586, 0.52109586, 0.52109586,
                     0.52109586]) #typo made this the average between the max and mean differences, rather than mean difference

# BUG

Whenever ATLA was resumed witha trained agent, the adversary was not given additional ts before training, so never changed it's policy. This explains why the only successful training involved multiple iterations. It also explains why the learning curves are different from the literature. Perhaps muuuuuuuch longer alternations are required  for a similar effect, so the agent recovers performance.

##### 2-10-13
- N_ALT = 10
- AGENT_ALT_EPISODES = 20 #PPO take longer to converge than SAC
- ADV_ALT_EPISODES = 20 #SAC adaquately converges in this time
- PRE_TRAINING_EPISODES = 0
- PRE_TRAINED_AGENT = 'Models\Victim\SAC_citylearn_challenge_2022_phase_2_Building_6_default_rwd_MARLISA_hyperparams_500.zip'
- PRE_TRAINED_ADV = None
- MASK=np.arange(6,31) #only features 7-31 will be perturbed, temporal features left alone
- PERTURBATION_SCALE = 1
- PERTURBATION_SPACE = MEAN_DIFF*PERTURBATION_SCALE

KPIs were above one with scores in the ~-7200, trying more alts

##### 2-11-21

- AGENT_ALT_EPISODES = 20 #PPO take longer to converge than SAC
- ADV_ALT_EPISODES = 20 #SAC adaquately converges in this time
- PRE_TRAINING_EPISODES = 0
- PRE_TRAINED_AGENT = 'Models\ATLAc\2-10-13 SAC agent 20 alts over 500+200.zip'
- PRE_TRAINED_ADV = 'Models\ATLAc\2-10-13 SAC agent 20 alts over 500+200.zip'
- MASK=np.arange(6,31) #only features 7-31 will be perturbed, temporal features left alone
- PERTURBATION_SCALE = 1
- PERTURBATION_SPACE = MEAN_DIFF*PERTURBATION_SCALE

Trying an additional 10 alts -> KPIs convergered to ~1, meaning the perturbations were too large?

##### 2-12-9

- N_ALT = 10
- AGENT_ALT_EPISODES = 20 #PPO take longer to converge than SAC
- ADV_ALT_EPISODES = 20 #SAC adaquately converges in this time
- PRE_TRAINING_EPISODES = 0
- PRE_TRAINED_AGENT = r'Models\Victim\SAC_citylearn_challenge_2022_phase_2_Building_6_default_rwd_MARLISA_hyperparams_500.zip'
- PRE_TRAINED_ADV = None
- MASK=np.arange(6,31) #only features 7-31 will be perturbed, temporal features left alone
- PERTURBATION_SCALE = 0.5
- PERTURBATION_SPACE = MEAN_DIFF*PERTURBATION_SCALE

Same score as previous

##### 2-13-7 -> 2-14-9 after GPU driver crash

Previous may not have convergered in 20 episodes during the rebound, so we give the agent more training episodes, but fewer alts.

- N_ALT = 7
- AGENT_ALT_EPISODES = 30 #PPO take longer to converge than SAC
- ADV_ALT_EPISODES = 20 #SAC adaquately converges in this time
- PRE_TRAINING_EPISODES = 0
- PRE_TRAINED_AGENT = r'Models\Victim\SAC_citylearn_challenge_2022_phase_2_Building_6_default_rwd_MARLISA_hyperparams_500.zip'
- PRE_TRAINED_ADV = None
- MASK=np.arange(6,31) #only features 7-31 will be perturbed, temporal features left alone
- PERTURBATION_SCALE = 0.5
- PERTURBATION_SPACE = MEAN_DIFF*PERTURBATION_SCALE

##### 2-15-4

See if agent from 2-14-9 continues to improve after 2 more alts

- N_ALT = 2
- AGENT_ALT_EPISODES = 30 #PPO take longer to converge than SAC
- ADV_ALT_EPISODES = 20 #SAC adaquately converges in this time
- PRE_TRAINING_EPISODES = 0
- PRE_TRAINED_AGENT = r'Models\ATLAc\2-14-9 SAC agent 30 alts over 500+210.zip'
- PRE_TRAINED_ADV = r'Models\ATLAc\2-14-9 SAC adversary 20 alts over 140.zip'
- MASK=np.arange(6,31) #only features 7-31 will be perturbed, temporal features left alone
- PERTURBATION_SCALE = 0.5
- PERTURBATION_SPACE = MEAN_DIFF*PERTURBATION_SCALE

##### 2-15-8

Last trial improved performance, so we'll try a few more alts

- N_ALT = 4
- AGENT_ALT_EPISODES = 30 #PPO take longer to converge than SAC
- ADV_ALT_EPISODES = 20 #SAC adaquately converges in this time
- PRE_TRAINING_EPISODES = 0
- PRE_TRAINED_AGENT = r'Models\ATLAc\2-15-4 SAC agent 30 alts over 710+60.zip'
- PRE_TRAINED_ADV = r'Models\ATLAc\2-15-4 SAC agent 30 alts over 710+60.zip'
- MASK=np.arange(6,31) #only features 7-31 will be perturbed, temporal features left alone
- PERTURBATION_SCALE = 0.5
- PERTURBATION_SPACE = MEAN_DIFF*PERTURBATION_SCALE

##### 2-15-15 (best, but KPIs near 1)

Last trial improved performance, so we'll try a few more alts again

- N_ALT = 4
- AGENT_ALT_EPISODES = 30 #PPO take longer to converge than SAC
- ADV_ALT_EPISODES = 20 #SAC adaquately converges in this time
- PRE_TRAINING_EPISODES = 0
- PRE_TRAINED_AGENT = r'Models\ATLAc\2-15-8 SAC agent 30 alts over 770+120.zip'
- PRE_TRAINED_ADV = r'Models\ATLAc\2-15-8 SAC adversary 20 alts over 80.zip'
- MASK=np.arange(6,31) #only features 7-31 will be perturbed, temporal features left alone
- PERTURBATION_SCALE = 0.5
- PERTURBATION_SPACE = MEAN_DIFF*PERTURBATION_SCALE

Small gains, perhaps restarting longer agent training will help

##### 2-15-22

- N_ALT = 10
- AGENT_ALT_EPISODES = 50*
- ADV_ALT_EPISODES = 20 
- PRE_TRAINING_EPISODES = 0
- PRE_TRAINED_AGENT = r'Models\Victim\SAC_citylearn_challenge_2022_phase_2_Building_6_default_rwd_MARLISA_hyperparams_500.zip'*
- PRE_TRAINED_ADV = None
- MASK=np.arange(6,31) #only features 7-31 will be perturbed, temporal features left alone
- PERTURBATION_SCALE = 0.5
- PERTURBATION_SPACE = MEAN_DIFF*PERTURBATION_SCALE

At half way the agent seems to have converged to a nil-action policy, and the extra training episodes might be detrimental. Unlike the previous trial, there is no upward trend (or small non-linear improvements). Stopped after 5 iterations as performance became flat.  Decrease perturbation space?

##### 2-16-18

Increasing agent training did not lead to better performance, so the perturbation size has been reduce for this attempt.

- N_ALT = 10
- AGENT_ALT_EPISODES = 30 #PPO take longer to converge than SAC
- ADV_ALT_EPISODES = 20 #SAC adaquately converges in this time
- PRE_TRAINING_EPISODES = 0
- PRE_TRAINED_AGENT = r'Models\Victim\SAC_citylearn_challenge_2022_phase_2_Building_6_default_rwd_MARLISA_hyperparams_500.zip'
- PRE_TRAINED_ADV = None
- MASK=np.arange(6,31) #only features 7-31 will be perturbed, temporal features left alone
- PERTURBATION_SCALE = 0.25
- PERTURBATION_SPACE = MEAN_DIFF*PERTURBATION_SCALE

Eval score still rapidly drops to -7000, adversary may still have too large a budget. Strangly this resulted in worse performance than 2-14-9/2-15-8 (which had twice the perturbation size), so the issue may not be perturbation size? Those trials had score near 1, so that may be the result of too large perturbations

##### 2-17-21

See if we can get agent KPIs above 1 with smaller perturbations

- N_ALT = 10
- AGENT_ALT_EPISODES = 30 #PPO take longer to converge than SAC
- ADV_ALT_EPISODES = 20 #SAC adaquately converges in this time
- PRE_TRAINING_EPISODES = 0
- PRE_TRAINED_AGENT = r'Models\Victim\SAC_citylearn_challenge_2022_phase_2_Building_6_default_rwd_MARLISA_hyperparams_500.zip'
- PRE_TRAINED_ADV = None
- MASK=np.arange(6,31) #only features 7-31 will be perturbed, temporal features left alone
- PERTURBATION_SCALE = 0.125*
- PERTURBATION_SPACE = MEAN_DIFF*PERTURBATION_SCALE

Samller B(s) resulted in smaller adv regret, but agent failed to improve from the reward floor.

##### 2-19-10 (cont 2-15-15)

With no more promising agents, we'll continue where our best agent at (2-15-15) left off

- N_ALT = 4
- AGENT_ALT_EPISODES = 30
- ADV_ALT_EPISODES = 20
- PRE_TRAINING_EPISODES = 0
- PRE_TRAINED_AGENT = r'Models\ATLAc\2-15-15 SAC agent 30 alts over 890+120.zip'
- PRE_TRAINED_ADV = r'Models\ATLAc\2-15-15 SAC adversary 20 alts over 80.zip'
- MASK=np.arange(6,31) #only features 7-31 will be perturbed, temporal features left alone
- PERTURBATION_SCALE = 0.5
- PERTURBATION_SPACE = MEAN_DIFF*PERTURBATION_SCALE

Electricity consumption is now below 1, may improve with further training. Performance is better under attack than unperturbed -> agent migh be over fitting as adv is not changing

##### 2-19-19 -> 2-19-22 (due to power bump)

since reloading agents to extend training 2-14-9 was not updating the adv due to a bug, we'll retraing from there with the predicted behaviour. Previous agents seem to overfit with the single adv. 

- N_ALT = 5
- AGENT_ALT_EPISODES = 30
- ADV_ALT_EPISODES = 20
- PRE_TRAINING_EPISODES = 0
- PRE_TRAINED_AGENT = r'Models\ATLAc\2-14-9 SAC agent 30 alts over 500+210.zip'
- PRE_TRAINED_ADV = r'Models\ATLAc\2-14-9 SAC adversary 20 alts over 140.zip'
- MASK=np.arange(6,31) #only features 7-31 will be perturbed, temporal features left alone
- PERTURBATION_SCALE = 0.5
- PERTURBATION_SPACE = MEAN_DIFF*PERTURBATION_SCALE
- TRAIN_LOADED_ADV = True #bug prevented adversary from training when loaded from storage, this allows this aspect to be enabled for continuing previous trials


##### 2-20-7

Error peristed, so rerunning the previous trial with fix

- N_ALT = 5
- AGENT_ALT_EPISODES = 30
- ADV_ALT_EPISODES = 20
- PRE_TRAINING_EPISODES = 0
- PRE_TRAINED_AGENT = r'Models\ATLAc\2-14-9 SAC agent 30 alts over 500+210.zip'
- PRE_TRAINED_ADV = r'Models\ATLAc\2-14-9 SAC adversary 20 alts over 140.zip'
- MASK=np.arange(6,31) #only features 7-31 will be perturbed, temporal features left alone
- PERTURBATION_SCALE = 0.5
- PERTURBATION_SPACE = MEAN_DIFF*PERTURBATION_SCALE
- TRAIN_LOADED_ADV = True #bug prevented adversary from training when loaded from storage, this allows this aspect to be enabled for continuing previous trials

performance is reducing, we can try again with longer agent training (learning I POMDP as the agent is harder than the MDP for the adversary) or see if continuing with the same adv as per 2-19-10 yields a usable agent in a clean setting. Maybe train it against a new adversary? Note that the adversary trains after the agent, so it's facing an unseen adversary when training starts

#####  2-20-21
continuing from 2-19-10 (trace began with 2-15-15) with the adversary updating and extended number of episodes for the agent to train.

- N_ALT = 2
- AGENT_ALT_EPISODES = 100*
- ADV_ALT_EPISODES = 20
- PRE_TRAINING_EPISODES = 0
- PRE_TRAINED_AGENT = r'Models\ATLAc\2-19-10 SAC agent 30 alts over 1010+120.zip'
- PRE_TRAINED_ADV = r'Models\ATLAc\2-19-10 SAC adversary 20 alts over 80.zip'
- MASK=np.arange(6,31) #only features 7-31 will be perturbed, temporal features left alone
- PERTURBATION_SCALE = 0.5
- PERTURBATION_SPACE = MEAN_DIFF*PERTURBATION_SCALE
- TRAIN_LOADED_ADV = True #bug prevented adversary from training when loaded from storage, this allows this aspect to be enabled for continuing previous 

Performance dropped and never recovered after the adversary was updated, the Agent need more time to recover before presented with a new adversary

##### 2-21-10

Since and updated adversary collapsed performance even after 100 epsiodes of training togther (in 2-20-21), more time is required for the agent to adapt to a new adversary. Start again from 2-14-9, but with more training time for the agent? I'm worried the following agent overfit given how much a new adversary tanks performance. We'll start from 2-14-9, and training begins with a new adversary. THe agent's trining time will be increased (POMDPs are harder than MDPs)

- N_ALT = 2
- AGENT_ALT_EPISODES = 100
- ADV_ALT_EPISODES = 20
- PRE_TRAINING_EPISODES = 0
- PRE_TRAINED_AGENT = r'Models\ATLAc\2-14-9 SAC agent 30 alts over 500+210.zip'
- PRE_TRAINED_ADV = r'Models\ATLAc\2-14-9 SAC adversary 20 alts over 140.zip'
- MASK=np.arange(6,31) #only features 7-31 will be perturbed, temporal features left alone
- PERTURBATION_SCALE = 0.5
- PERTURBATION_SPACE = MEAN_DIFF*PERTURBATION_SCALE
- TRAIN_LOADED_ADV = True #bug prevented adversary from training when loaded from storage, this allows this aspect to be enabled for continuing previous trials

In [4]:
R_EXP = 3 #for norm distance reward

N_ALT = 2
AGENT_ALT_EPISODES = 100
ADV_ALT_EPISODES = 20
PRE_TRAINING_EPISODES = 0
PRE_TRAINED_AGENT = r'Models\ATLAc\2-14-9 SAC agent 30 alts over 500+210.zip'
PRE_TRAINED_ADV = r'Models\ATLAc\2-14-9 SAC adversary 20 alts over 140.zip'
MASK=np.arange(6,31) #only features 7-31 will be perturbed, temporal features left alone
PERTURBATION_SCALE = 0.5
PERTURBATION_SPACE = MEAN_DIFF*PERTURBATION_SCALE
TRAIN_LOADED_ADV = True #bug prevented adversary from training when loaded from storage, this allows this aspect to be enabled for continuing previous trials

EVAL_PER_ALT = 1
ADV_TOTAL_EPISODES = ADV_ALT_EPISODES*N_ALT
AGENT_TOTAL_EPISODES = AGENT_ALT_EPISODES*N_ALT + PRE_TRAINING_EPISODES


Define SB3 environments, note the the eval and training environments must be difference objects

In [5]:
kwargs = dict(
    schema=DataSet.get_schema(DATASET_NAME),
    T=None #this was supposed to make evaluations shorter, but does not work... never passed it in lol
)
agent_env = utils.make_continuous_env(schema=DataSet.get_schema(DATASET_NAME),  
                        seed=0)

agent_eval_env = utils.make_continuous_env(schema=DataSet.get_schema(DATASET_NAME),  
                        seed=42)

adv_env = utils.make_continuous_env(schema=DataSet.get_schema(DATASET_NAME),  
                        seed=0)

adv_eval_env = utils.make_continuous_env(schema=DataSet.get_schema(DATASET_NAME),  
                        seed=42)
if kwargs['T'] is not None:
    print('T should be None unless this is a test')

In [6]:
T = agent_env.time_steps - 1
print(f'Each episode is {T} timesteps')

Each episode is 8759 timesteps


Define agent (could load/save pretrained agent)

In [7]:
if PRE_TRAINED_AGENT is None:
    policy_kwargs = dict(net_arch=[256, 256])
    agent = SAC('MlpPolicy', 
                agent_env,
                device=DEVICE,
                policy_kwargs=policy_kwargs,
                tensorboard_log=LOG_DIR,
                verbose=VERBOSITY,
                )
    print('new agent defined')
else:
    agent = SAC.load(path=PRE_TRAINED_AGENT,
                     env=agent_env,
                     device=DEVICE,
                     tensorboard_log=LOG_DIR,
                     verbose=VERBOSITY,
                     print_system_info=True,
                     #force_reset=False, #default is true for continued training ref: https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html#stable_baselines3.ppo.PPO.load
                     )
    print('agent loaded from storage')

== CURRENT SYSTEM INFO ==
- OS: Windows-10-10.0.19045-SP0 10.0.19045
- Python: 3.10.12
- Stable-Baselines3: 1.8.0
- PyTorch: 1.12.1
- GPU Enabled: True
- Numpy: 1.23.5
- Gym: 0.21.0

== SAVED MODEL SYSTEM INFO ==
- OS: Windows-10-10.0.19045-SP0 10.0.19045
- Python: 3.10.12
- Stable-Baselines3: 1.8.0
- PyTorch: 1.12.1
- GPU Enabled: True
- Numpy: 1.23.5
- Gym: 0.21.0



c:\Users\Broda-Milian\anaconda3\envs\CityLearnART\lib\site-packages\gym\spaces\box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


agent loaded from storage


The number of timesteps and agent has trained is non-zero when loaded from storage, this must be added to the pause and total timesteps so training is not prematurely aborted

In [8]:
agent_n_ts = agent.num_timesteps

In [9]:
now = datetime.now()
dtg = f'{now.month}-{now.day}-{now.hour}'

Name contains RL algorithm, episodes per alternation and total episodes, followed by a the date-time with hour precision

In [10]:
agent_name = f'{dtg} {agent.__class__.__name__} agent {AGENT_ALT_EPISODES} alts over '
if PRE_TRAINING_EPISODES > 0:
    agent_name += f'{PRE_TRAINING_EPISODES}+'
else:
    agent_name += f'{agent_n_ts//T}+'
agent_name += f'{AGENT_ALT_EPISODES*N_ALT}'


Agent pre-training

In [11]:
if PRE_TRAINING_EPISODES > 0:
    print(f'Pre-training for {PRE_TRAINING_EPISODES*T} timesteps ({PRE_TRAINING_EPISODES} episodes)')
    agent.learn(total_timesteps=AGENT_TOTAL_EPISODES*T + agent_n_ts,
                callback=[EvalCallback(Monitor(agent_eval_env),
                                       eval_freq=PRE_TRAINING_EPISODES//EVAL_PER_ALT*T,
                                       verbose=VERBOSITY),
                          ATLA.HParamCallback(),
                          ATLA.PauseOnStepCallback(PRE_TRAINING_EPISODES*T + agent_n_ts)], #stops training before ts budget expended
                tb_log_name=agent_name,
                reset_num_timesteps=False, #allows training to continue where it left off
                progress_bar=True,
                log_interval=1 #start logging after first epsiode, for debugging
                )
    print(f'Agent pretrained for {agent.num_timesteps - agent_n_ts} timesteps, or {(agent.num_timesteps - agent_n_ts)/T} episodes')
else:
    print('No pretraining specified')

No pretraining specified


Define adversary's reward

In [12]:
rwd = ATLA.NormScaleReward(adv_env, 
                            np.inf,
                            exp=R_EXP,
                            )

Define an adv action space in [-1,1] for ATLA.BScaledSumPrevProj, which scale a maximum perturbation

In [13]:
normalized_a_space = gym.spaces.Box(low=-1*np.ones(MASK.shape),
                                    high=np.ones(MASK.shape),
                                    dtype='float32',)

##### Parameterize the B function
- The adversary adds a bounded perturbation to the current observation with B(s) as BScaledSum
- 

Max perturbation reduced to mean diff devided by 2

In [14]:
B_params = dict(
    #clip_bound=np.ones(agent_env.observation_space.shape)*0.33,
    max_perturbation=np.ones(MASK.shape)*PERTURBATION_SPACE[MASK]
                  )

Define adversary's environment

In [15]:
kwargs = dict(
    #adv_reward=rwd, #use default negative agent reward
    victim=agent,
    B=ATLA.BScaledSum,
    action_space=normalized_a_space, #[-1,1] for scaled B defined above
    feature_mask=MASK, 
    B_kwargs=B_params,
)
adv_eval_env = ATLA.AdversaryATLAWrapper(env=adv_eval_env, **kwargs)
adv_eval_env = Monitor(adv_eval_env)

adv_env = ATLA.AdversaryATLAWrapper(env=adv_env, **kwargs)


In [16]:
#check_env(adv_env,)

Define adversary

In [17]:
if PRE_TRAINED_ADV is None:
    policy_kwargs = dict(net_arch=[256, 256])
    adversary = SAC('MlpPolicy', 
            Monitor(adv_env),
            device=DEVICE,
            policy_kwargs=policy_kwargs,
            tensorboard_log=LOG_DIR,
            verbose=VERBOSITY,
            )
    print('new adversary defined')
else:
    adversary = SAC.load(path=PRE_TRAINED_ADV,
                     env=adv_env,
                     device=DEVICE,
                     tensorboard_log=LOG_DIR,
                     verbose=VERBOSITY,
                     print_system_info=True,
                     #force_reset=False, #default is true for continued training ref: https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html#stable_baselines3.ppo.PPO.load
                     )
    print('adversary loaded from storage')
if TRAIN_LOADED_ADV:
    adv_n_ts = adversary.num_timesteps
else: #legacy behaviour for pre 2-19-10 trials
    adv_n_ts = 0


== CURRENT SYSTEM INFO ==
- OS: Windows-10-10.0.19045-SP0 10.0.19045
- Python: 3.10.12
- Stable-Baselines3: 1.8.0
- PyTorch: 1.12.1
- GPU Enabled: True
- Numpy: 1.23.5
- Gym: 0.21.0

== SAVED MODEL SYSTEM INFO ==
- OS: Windows-10-10.0.19045-SP0 10.0.19045
- Python: 3.10.12
- Stable-Baselines3: 1.8.0
- PyTorch: 1.12.1
- GPU Enabled: True
- Numpy: 1.23.5
- Gym: 0.21.0

adversary loaded from storage


In [18]:
adv_name = f'{dtg} {adversary.__class__.__name__} adversary {ADV_ALT_EPISODES} alts over {ADV_TOTAL_EPISODES}'

Define the adversary's perturbation function for the victim environment. We use a function which applies the corresponding B(s) to the adversary's prediction 

In [19]:
perturbation = ATLA.sb3_perturbation(adversary,)

Wrap agent's environments for ATLA

In [20]:
#perturbation=adversary.predict
agent_env = ATLA.VictimATLAWrapper(agent_env,
                                   perturbation,)
agent_eval_env = ATLA.VictimATLAWrapper(agent_eval_env,
                                        perturbation,)
agent_eval_env = Monitor(agent_eval_env)


In [21]:
check_env(agent_env)

c:\Users\Broda-Milian\anaconda3\envs\CityLearnART\lib\site-packages\stable_baselines3\common\env_checker.py:384: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn(


replace pre-training environment with ATLA environment

In [22]:
agent.set_env(agent_env)

Define ATLA evaluation callbacks

In [23]:
kwargs = dict(
    verbose=1 #print scores
)

adv_eval_callback = EvalCallback(adv_eval_env, 
                                 eval_freq=ADV_ALT_EPISODES//EVAL_PER_ALT*T, 
                                 **kwargs)
agent_eval_callback = EvalCallback(agent_eval_env,
                                   eval_freq=AGENT_ALT_EPISODES//EVAL_PER_ALT*T,
                                   **kwargs)

Conduct ATLA. Note:
- the agents are not reset between iterations, this prevents attributes like scaled exploration and learning rates from resetting.
- A callback pauses training after a number of episodes has elapsed but before the max training budget is reached (does this work better than resetting?). 
- Adversary was originally trained first, so that the agent would start training against a trained adversary. However, the agent is trained first in the ATLA paper, implying it first faces a randomly initialized adversary. Try reversing them?

In [24]:
kwargs = dict(
    reset_num_timesteps=False, #allows training to continue where it left off between .learn() calls
    progress_bar=False, # progress bar really slows cell execution
    log_interval=1 #start logging after first epsiode, useful for debugging
)
print(f'ATLA for {N_ALT}')
for alt in range(N_ALT):
    #first trial had the agent train first, so these were reversed
    agent.learn(total_timesteps=AGENT_TOTAL_EPISODES*T + agent_n_ts,
                callback=[agent_eval_callback,
                          ATLA.HParamCallback(),
                          ATLA.PauseOnStepCallback(T*(AGENT_ALT_EPISODES*(1 + alt) + PRE_TRAINING_EPISODES) + agent_n_ts)],
                    tb_log_name=agent_name,
                    **kwargs)
    print(f'Agent trained has for {agent.num_timesteps} ts ({agent.num_timesteps/T} episodes) up to iteration {alt}')
    
    if SAVE_DIR is not None:
        try:
            agent.save(SAVE_DIR + agent_name)
            print('agent progress saved')
        except:
            print(f'error saving to {SAVE_DIR}')
    
    adversary.learn(total_timesteps=ADV_TOTAL_EPISODES*T + adv_n_ts, #missing +adv_n_ts, so the adversary is not actually training when loaded...
                    callback=[adv_eval_callback,
                              ATLA.AdvDistanceTensorboardCallback(),
                              ATLA.HParamCallback(),
                              ATLA.PauseOnStepCallback(ADV_ALT_EPISODES*(1 + alt)*T + adv_n_ts)], #pauses training before the max ts, updates each iter
                    tb_log_name=adv_name,
                    **kwargs)
    print(f'Adversary has trained for {adversary.num_timesteps} ts ({adversary.num_timesteps/T} episodes) up to iteration {alt}')
    
    if SAVE_DIR is not None:
        try:
            adversary.save(SAVE_DIR + adv_name)
            print('adversary progress saved')

        except:
            print(f'error saving to {SAVE_DIR}')
    

ATLA for 2
Eval num_timesteps=7094790, episode_reward=-6913.42 +/- 0.00
Episode length: 8759.00 +/- 0.00
New best mean reward!
Agent trained has for 7094790 ts (810.0 episodes) up to iteration 0
agent progress saved
Eval num_timesteps=1401440, episode_reward=8195.99 +/- 0.00
Episode length: 8759.00 +/- 0.00
New best mean reward!
Adversary has trained for 1401440 ts (160.0 episodes) up to iteration 0
adversary progress saved
Eval num_timesteps=7970690, episode_reward=-7184.25 +/- 0.00
Episode length: 8759.00 +/- 0.00
Agent trained has for 7970690 ts (910.0 episodes) up to iteration 1
agent progress saved
Eval num_timesteps=1576620, episode_reward=8341.98 +/- 0.00
Episode length: 8759.00 +/- 0.00
New best mean reward!
Adversary has trained for 1576620 ts (180.0 episodes) up to iteration 1
adversary progress saved


Save models

In [25]:
if SAVE_DIR is not None:
    agent.save(SAVE_DIR + agent_name)
    adversary.save(SAVE_DIR + adv_name)